In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import regex
import re
import os
from Bio import SeqIO
import Bio
from Bio import motifs
from Bio import pairwise2 
from Bio.pairwise2 import format_alignment
import collections
import itertools


Install package inside Jupter Notebook

In [13]:
import sys
!{sys.executable} -m pip install openpyxl


     -------------------------------------- 249.8/249.8 kB 3.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: C:\Users\shangl02\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [42]:

# set up working directory
os.chdir(r'X:\users\shangl02\Test\ampliCan.test2\AS15\fastq.merge')
directory = os.getcwd()
#filenames = os.listdir(directory, '*.fastq')
filenames=[]
for file in os.listdir(directory):
    if file.endswith(".fastq"):
        filenames.append(file)
print(filenames)




['AS15-01_S1.fastq', 'AS15-02_S2.fastq', 'AS15-03_S3.fastq', 'AS15-04_S4.fastq', 'AS15-05_S5.fastq', 'AS15-06_S6.fastq', 'AS15-07_S7.fastq', 'AS15-08_S8.fastq', 'AS15-09_S9.fastq', 'AS15-10_S10.fastq', 'AS15-11_S11.fastq', 'AS15-12_S12.fastq', 'AS15-13_S13.fastq', 'AS15-14_S14.fastq', 'AS15-15_S15.fastq', 'AS15-16_S16.fastq', 'AS15-17_S17.fastq', 'AS15-18_S18.fastq', 'AS15-19_S19.fastq', 'AS15-20_S20.fastq', 'AS15-21_S21.fastq', 'AS15-22_S22.fastq', 'AS15-23_S23.fastq', 'AS15-24_S24.fastq', 'AS15-25_S25.fastq', 'AS15-26_S26.fastq', 'AS15-27_S27.fastq', 'AS15-28_S28.fastq', 'AS15-29_S29.fastq', 'AS15-30_S30.fastq', 'AS15-31_S31.fastq', 'AS15-32_S32.fastq', 'AS15-33_S33.fastq', 'AS15-34_S34.fastq', 'AS15-35_S35.fastq', 'AS15-36_S36.fastq', 'AS15-37_S37.fastq', 'AS15-38_S38.fastq', 'AS15-39_S39.fastq', 'AS15-40_S40.fastq', 'AS15-41_S41.fastq', 'AS15-42_S42.fastq', 'AS15-43_S43.fastq', 'AS15-44_S44.fastq', 'AS15-45_S45.fastq', 'AS15-46_S46.fastq', 'AS15-47_S47.fastq', 'AS15-48_S48.fastq', 

In [43]:

# generate metadata dataframe from submitted sequence information sheet including ‘Sample #Number’, ‘Guide Sequence’ and ‘Full Amplicon’. Add two columns indicating reverse complement #sequences
meta = pd.read_excel(r'X:\users\shangl02\Test\ampliCan.test2\AS15\AS15_submissionform for Eva.xlsx')
meta['RC_Full Amplicon'] = [Bio.Seq.reverse_complement(seq) for seq in meta['Full Amplicon']]
meta['RC_Guide Sequence'] = [Bio.Seq.reverse_complement(seq) if isinstance(seq, str) else seq for seq in meta['Guide Sequence']]

#Add a column to the metadata dataframe indicating the orientation of guide
guide_in_sense = []
for index, row in meta.iterrows():
    if isinstance(row['Guide Sequence'], float):
        check = 'NA'
    elif row['Guide Sequence'] in row['Full Amplicon']:
        check = True
    elif row['Guide Sequence'] in row['RC_Full Amplicon']:
        check = False
    guide_in_sense.append(check)
meta['guide_in_sense'] = guide_in_sense

print(meta)

   Sample Number    Sample Name  Experiment #  \
0        AS15_01  BCL_GFP/500_1             1   
1        AS15_02  BCL_GFP/500_2             1   
2        AS15_03  BCL_GFP/500_3             1   
3        AS15_04  BCL_IDT/500_1             1   
4        AS15_05  BCL_IDT/500_2             1   
..           ...            ...           ...   
87       AS15_88     HBG_sg44_3             2   
88       AS15_89     HBG_sg45_3             2   
89       AS15_90     HBG_sg46_3             2   
90       AS15_91     HBG_sg50_3             2   
91       AS15_92     HBG_sg30_3             2   

                                        Full Amplicon  \
0   TCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCAGTGCAAAGTC...   
1   TCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCAGTGCAAAGTC...   
2   TCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCAGTGCAAAGTC...   
3   TCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCAGTGCAAAGTC...   
4   TCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCAGTGCAAAGTC...   
..                                                ...   
87  TCGTCGGC

In [44]:
#Add two columns indicating the 10bp upstream and downstream of protospacer, respectively
flank1 = {}
flank2 = {}
for index, row in meta.iterrows():
    if isinstance(row['Guide Sequence'], float):
        continue
    if row['Guide Sequence'] in row['Full Amplicon']:
        flank1[index] = row['Full Amplicon'].split(row['Guide Sequence'])[0][-10:]
        flank2[index] = row['Full Amplicon'].split(row['Guide Sequence'])[1][:10]       
    elif row['Guide Sequence'] in row['RC_Full Amplicon']:
        flank1[index] = row['Full Amplicon'].split(row['RC_Guide Sequence'])[0][-10:]
        flank2[index] = row['Full Amplicon'].split(row['RC_Guide Sequence'])[1][:10]
flank_df = pd.DataFrame([flank1, flank2], index = ['flank1', 'flank2']).T
meta1 = meta.merge(flank_df, left_index = True, right_index = True)

In [45]:
#Main base calling function
def basecall(directory, filename, guide, flank1, flank2, guide_in_sense):
    seqs = {}
    seq_parse = list(SeqIO.parse(os.path.join(directory, filename), 'fastq'))
    for record in seq_parse:
        recordqual = [x > 31 for x in record.letter_annotations['phred_quality']]
        if float(sum(recordqual)) / float(len(recordqual)) > 0.5:
            recordseq = ''.join([y if x else 'N' for (x,y) in zip(recordqual, record.seq)])
            split1 = recordseq.split(flank1)
            if len(split1) == 2:
                split2 = split1[1].split(flank2)[0]
                if len(split2) == 20:
                    if guide_in_sense == False:
                        seqs[record.id] = Bio.Seq.reverse_complement(split2)
                    else:
                        seqs[record.id] = split2
    frame = pd.DataFrame(list(seqs.items()), columns = ['ID', 'spacer'])
    frame1 = frame[['N' not in x for x in frame.spacer]]
    Motif = motifs.create(frame1.spacer.values)
    raw = pd.DataFrame(Motif.counts, index = [str(i+1) for i in range(20)]).transpose()
    normalized = raw / raw.sum(axis = 0) * 100
    normalized = normalized.round(2)
    guide_split = [*guide]
    per_base = pd.concat([pd.DataFrame(guide_split, index = [str(i+1) for i in range(20)]).transpose(), normalized])
    counts = frame1.groupby('spacer').count().sort_values('ID', ascending = False)
    counts['percentage'] = counts['ID'] / counts['ID'].sum() * 100
    return [per_base, counts]


In [46]:
print(meta1)

   Sample Number    Sample Name  Experiment #  \
0        AS15_01  BCL_GFP/500_1             1   
1        AS15_02  BCL_GFP/500_2             1   
2        AS15_03  BCL_GFP/500_3             1   
3        AS15_04  BCL_IDT/500_1             1   
4        AS15_05  BCL_IDT/500_2             1   
..           ...            ...           ...   
87       AS15_88     HBG_sg44_3             2   
88       AS15_89     HBG_sg45_3             2   
89       AS15_90     HBG_sg46_3             2   
90       AS15_91     HBG_sg50_3             2   
91       AS15_92     HBG_sg30_3             2   

                                        Full Amplicon  \
0   TCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCAGTGCAAAGTC...   
1   TCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCAGTGCAAAGTC...   
2   TCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCAGTGCAAAGTC...   
3   TCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCAGTGCAAAGTC...   
4   TCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCAGTGCAAAGTC...   
..                                                ...   
87  TCGTCGGC

In [47]:
#Run every fastq file in working directory
res = {}
for index,row in meta1.iterrows():
    for file in filenames:
        #if row['Sample Number'] in file:
        if file.replace('-','_').startswith(row['Sample Number']):
            print(file)
            res[file] = basecall(directory, file, row['Guide Sequence'], \
                                 row['flank1'], row['flank2'], row['guide_in_sense'])

AS15-01_S1.fastq
AS15-02_S2.fastq
AS15-03_S3.fastq
AS15-04_S4.fastq
AS15-05_S5.fastq
AS15-06_S6.fastq
AS15-07_S7.fastq
AS15-08_S8.fastq
AS15-09_S9.fastq
AS15-10_S10.fastq
AS15-11_S11.fastq
AS15-12_S12.fastq
AS15-13_S13.fastq
AS15-14_S14.fastq
AS15-15_S15.fastq
AS15-16_S16.fastq
AS15-17_S17.fastq
AS15-18_S18.fastq
AS15-19_S19.fastq
AS15-20_S20.fastq
AS15-21_S21.fastq
AS15-22_S22.fastq
AS15-23_S23.fastq
AS15-24_S24.fastq
AS15-25_S25.fastq
AS15-26_S26.fastq
AS15-27_S27.fastq
AS15-28_S28.fastq
AS15-29_S29.fastq
AS15-30_S30.fastq
AS15-31_S31.fastq
AS15-32_S32.fastq


AttributeError: 'DataFrame' object has no attribute 'spacer'

In [ ]:

#there are two outputs from scripts implemented above
# 1. per base percentage of each nucleotide
 

# 2. Counts and percentage of each sequence pattern
 

#Get a single value of base editing efficiency from all samples
be_eff_all = {}
for key, value in res.items():
    sample = key[:7]
    guide = meta1.loc[meta1['Sample Number'] == sample, 'Guide Sequence'].tolist()[0]
    df = value[1][value[1].index != guide]
    eff = df['percentage'][0:5].sum()
    be_eff_all[key[:7]] = eff
be_eff_df = pd.DataFrame(be_eff_all.items(), columns = ['sample', 'base_editing_efficiency'])
#the output